In [52]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import matplotlib as mpl
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

mpl.rc('font',**{'family':'serif','serif':['serif']})
mpl.rc('text', usetex=True)
import fitsio as fio

import h5py

import copy

import xpipe.tools.catalogs as catalogs
import xpipe.paths as paths
import xpipe.xhandle.parbins as parbins
import xpipe.xhandle.xwrap as xwrap
import xpipe.tools.selector as selector
import xpipe.xhandle.shearops as shearops
import xpipe.xhandle.pzboost as pzboost
import xpipe.tools.y3_sompz as sompz
import xpipe.tools.mass as mass
import xpipe.tools.visual as visual


from importlib import reload
import pickle


import astropy.cosmology as cosmology
# this is just the default cosmology
cosmo = cosmology.FlatLambdaCDM(Om0=0.3, H0=70)

# we make sure the correct config file is loaded here, it will let us automatically now what type of files 
# were / will be produced, and where they will be placed
paths.update_params("/home/moon/vargatn/DES/PROJECTS/xpipe/settings/params_y3lwb-v02_meta.yml")

updating params from: /home/moon/vargatn/DES/PROJECTS/xpipe/settings/params_y3lwb-v02_meta.yml

***********************

running in full mode

***********************



# Testing output

In [2]:
reload(sompz)

<module 'xpipe.tools.y3_sompz' from '/home/moon/vargatn/DES/PROJECTS/xpipe/xpipe/tools/y3_sompz.py'>

In [3]:
main_file_path = "/e/ocean1/users/vargatn/DESY3/Y3_mastercat_03_31_20.h5"
src = sompz.sompz_reader(main_file_path)
src.build_lookup()

In [4]:
flist, flist_jk, rlist, rlist_jk = parbins.get_file_lists(paths.params, paths.dirpaths)

In [47]:
reload(shearops)

<module 'xpipe.xhandle.shearops' from '/home/moon/vargatn/DES/PROJECTS/xpipe/xpipe/xhandle/shearops.py'>

In [48]:
i = 3

In [50]:
reload(shearops)

<module 'xpipe.xhandle.shearops' from '/home/moon/vargatn/DES/PROJECTS/xpipe/xpipe/xhandle/shearops.py'>

In [51]:
mfac_opt2=1/(1+ms_opt2[i])

ACP = shearops.AutoCalibrateProfile([flist[i],], flist_jk[i], src, xlims=(0.01, 100), sbins=optsbins2[i])
ACP.get_profiles(ismeta=False, id_key="ID", z_key="Z", mfactor_sbins=mfac_opt2) #
# ACP_new.append(ACP)    

loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v07/lwb-y3-lowz+cmass_v07_lens_qbin-3.fits


In [31]:
ms_opt2=[np.array([-0.024,-0.037]),np.array([-0.024,-0.037]),np.array([-0.037,]),np.array([-0.037,])]
optsbins2=[(2,3),(2,3),(3,),(3,)]
# ACP_old = []
ACP_new = []
for i, fname in enumerate(flist):
    mfac_opt2=1/(1+ms_opt2[i])
    
    ACP = shearops.AutoCalibrateProfile([flist[i],], flist_jk[i], src, xlims=(0.01, 100), sbins=optsbins2[i])
    ACP.get_profiles(ismeta=False, id_key="ID", z_key="Z", mfactor_sbins=mfac_opt2) #
    ACP_new.append(ACP)    

loading source bin 2
loading source bin 3
/e/ocean1/users/vargatn/DES/DES_Y3A2_cluster/data/xshear_in/lwb-y3-lowz+cmass_v07/lwb-y3-lowz+cmass_v07_lens_qbin-0.fits
loading source bin 2


KeyboardInterrupt: 

In [21]:
target

,ID,index,RA,DEC,WSYS,Z,WEIGHT
0,1016761.0,50707,43.545575,1.200097,0.946144,0.620642,1.0
1,1016762.0,50708,43.628713,1.223640,0.940779,0.596930,1.0
2,1016764.0,50710,44.050143,1.230958,0.961983,0.614112,1.0
3,1016765.0,50711,44.170234,1.230144,0.934858,0.615811,1.0
4,1016766.0,50712,44.241018,1.197829,0.936552,0.540546,1.0
...,...,...,...,...,...,...,...
34218,1035140.0,69086,23.568225,2.668823,0.923135,0.558708,1.0
34219,1035141.0,69087,23.746751,2.757856,0.920309,0.621492,1.0
34220,1035142.0,69088,23.748853,2.598477,0.952662,0.615452,1.0
34221,1035143.0,69089,23.764014,2.670462,0.927212,0.611902,1.0


In [29]:
for i in np.arange(4):
    target = ACP_new[i].target
    zmean = np.average(target["Z"], weights=target["WSYS"])
    sc_p = []
    for val in optsbins2[i]:
        tmp = src.get_single_scinv(zmean, val)
        sc_p.append(tmp)
    sc_p = np.array(sc_p)
#     print(sc_p)
    
    sc_bin = []
    for val in optsbins2[i]:
        tmp = src.get_bin_scinv(target["Z"], val, weights=target["WSYS"])
        sc_bin.append(tmp)
    sc_bin = np.array(sc_bin)
#     print(sc_bin)  
    
#     print((sc_p.sum() / sc_bin.sum()))
    print((sc_p / sc_bin))


[1.02003447 1.01626519]
[1.00828595 1.00574526]
[0.99989163]
[1.01181695]


In [19]:
ACP._get_scinvs(z_key="Z")
val0 = copy.copy(ACP_new[0].scinvs)
# ACP._get_scinvs_bin(z_key="Z")
# val1 = copy.copy(ACP.scinvs)


ValueError: weights should have the same shape as a.

In [17]:
val0

array([0.00028487, 0.00030714])

In [14]:
ACP.target

,ID,index,RA,DEC,WSYS,Z,WEIGHT
0,1016761.0,50707,43.545575,1.200097,0.946144,0.620642,1.0
1,1016762.0,50708,43.628713,1.223640,0.940779,0.596930,1.0
2,1016764.0,50710,44.050143,1.230958,0.961983,0.614112,1.0
3,1016765.0,50711,44.170234,1.230144,0.934858,0.615811,1.0
4,1016766.0,50712,44.241018,1.197829,0.936552,0.540546,1.0
...,...,...,...,...,...,...,...
34218,1035140.0,69086,23.568225,2.668823,0.923135,0.558708,1.0
34219,1035141.0,69087,23.746751,2.757856,0.920309,0.621492,1.0
34220,1035142.0,69088,23.748853,2.598477,0.952662,0.615452,1.0
34221,1035143.0,69089,23.764014,2.670462,0.927212,0.611902,1.0
